In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import requests
import json
import time
import os
from dotenv import load_dotenv

In [ ]:
cities = [    # 10 German cities by population
         "Q64",    # Berlin: federal state, capital and largest city of Germany
         "Q1055",  # Hamburg: city and federal state in the North of Germany
         "Q1726",  # München: capital and most populous city of Bavaria, Germany
         "Q365",   # Köln: city in North Rhine-Westphalia, Germany
         "Q1794",  # Frankfurt am Main: city in Hesse, Germany
         "Q1022",  # Stuttgart: capital city of German federated state Baden-Württemberg
         "Q1718",  # Düsseldorf: capital city of the German federated state of North Rhine-Westphalia
         "Q1295",  # Dortmund: city in North Rhine-Westphalia, Germany
         "Q2066",  # Essen: city in North Rhine-Westphalia, Germany
         "Q2079",  # Leipzig: most populous city in the German state of Saxony
                  
              # 5 Austrian cities by population
         "Q1741",  # Wien: capital of and state in Austria
         "Q13298", # Graz: capital of Styria, Austria
         "Q41329", # Linz: capital city of Upper Austria, Austria
         "Q34713", # Salzburg: capital city of the federal state of Salzburg in Austria
         "Q1735"   # Innsbruck: capital of the state of Tyrol, Austria
         ]

In [ ]:
load_dotenv()

True

In [ ]:
API_key = os.getenv("city_api")

In [ ]:
# example call for city details
url = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities/Q60"
headers = {
    "X-RapidAPI-Key": API_key,
    "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
}
response = requests.request("GET", url, headers = headers)
response.json()

{'data': {'id': 123214,
  'wikiDataId': 'Q60',
  'type': 'CITY',
  'city': 'New York City',
  'name': 'New York City',
  'country': 'United States of America',
  'countryCode': 'US',
  'region': 'New York',
  'regionCode': 'NY',
  'elevationMeters': 10,
  'latitude': 40.712728,
  'longitude': -74.006015,
  'population': 8804190,
  'timezone': 'America__New_York',
  'distance': None,
  'deleted': False,
  'placeType': 'CITY'}}

In [ ]:
# loop through the list of cities, makes API calls and concatenate all city data into one dataframe
city_list = []
for city in cities:
    url = f"https://wft-geo-db.p.rapidapi.com/v1/geo/cities/{city}"
    headers = {
        "X-RapidAPI-Key": API_key,
        "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
    }
    response = requests.request("GET", url, headers = headers)
    time.sleep(2)
    city_df = pd.json_normalize(response.json())
    city_list.append(city_df)
cities_df = pd.concat(city_list, ignore_index = True)
cities_df = cities_df[["data.id", "data.wikiDataId", "data.city", "data.country", "data.elevationMeters", 
                       "data.latitude", "data.longitude", "data.population"]]
cities_df

In [ ]:
# rewrite the above as a function so it can be implemented elsewhere
def get_city_data(cities):
    city_list = []
    url = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities/Q60"
    headers = {
        "X-RapidAPI-Key": API_key,
        "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
    }
    test = requests.request("GET", url, headers = headers)
    time.sleep(1)
    if test.status_code >= 200 and test.status_code <= 299:
        for city in cities:
            url = f"https://wft-geo-db.p.rapidapi.com/v1/geo/cities/{city}"
            headers = {
                "X-RapidAPI-Key": API_key,
                "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
            }
            response = requests.request("GET", url, headers = headers)
            time.sleep(2)
            city_df = pd.json_normalize(response.json())
            city_list.append(city_df)
    else:
        return -1
    cities_df = pd.concat(city_list, ignore_index = True)
    return cities_df

In [ ]:
city_data = get_city_data(cities)
city_data

,data.id,data.wikiDataId,data.type,data.city,data.name,data.country,data.countryCode,data.region,data.regionCode,data.elevationMeters,data.latitude,data.longitude,data.population,data.timezone,data.distance,data.deleted,data.placeType
0,3453309,Q64,CITY,Berlin,Berlin,Germany,DE,Berlin,BE,None,52.516667,13.383333,3664088,Europe__Berlin,None,False,CITY
1,3453263,Q1055,CITY,Hamburg,Hamburg,Germany,DE,Hamburg,HH,None,53.550000,10.000000,1852478,Europe__Berlin,None,False,CITY
2,3449775,Q1726,CITY,Munich,Munich,Germany,DE,Bavaria,BY,519,48.134520,11.571000,1488202,Europe__Berlin,None,False,CITY
3,3044453,Q365,CITY,Cologne,Cologne,Germany,DE,North Rhine-Westphalia,NW,52,50.942222,6.957778,1083498,Europe__Berlin,None,False,CITY
4,3449552,Q1794,CITY,Frankfurt am Main,Frankfurt am Main,Germany,DE,Hesse,HE,112,50.113611,8.679722,764104,Europe__Berlin,None,False,CITY
5,3448426,Q1022,CITY,Stuttgart,Stuttgart,Germany,DE,Baden-Württemberg,BW,245,48.776111,9.177500,630305,Europe__Berlin,None,False,CITY
6,24131,Q1718,CITY,Düsseldorf,Düsseldorf,Germany,DE,North Rhine-Westphalia,NW,38,51.231144,6.772381,645923,Europe__Berlin,None,False,CITY
7,24928,Q1295,CITY,Dortmund,Dortmund,Germany,DE,North Rhine-Westphalia,NW,86,51.513889,7.465278,587696,Europe__Berlin,None,False,CITY
8,3025827,Q2066,CITY,Essen,Essen,Germany,DE,North Rhine-Westphalia,NW,116,51.450833,7.013056,582415,Europe__Berlin,None,False,CITY
9,3182259,Q2079,CITY,Leipzig,Leipzig,Germany,DE,Saxony,SN,113,51.340000,12.375000,597215,Europe__Berlin,None,False,CITY


In [ ]:
# set the dataframe to exclude unnecessary columns
city_data = city_data[["data.wikiDataId", "data.city", "data.country", "data.countryCode", "data.region", 
                       "data.elevationMeters", "data.latitude", "data.longitude", "data.population"]]

In [ ]:
# rename columns with more intuitive names and get rid of dots to avoid problems in MySQL
city_data.rename(columns = {"data.wikiDataId": "city_id",
                            "data.city": "city" ,
                            "data.country": "country",
                            "data.countryCode": "country_code",
                            "data.region": "region",
                            "data.elevationMeters": "elevation",
                            "data.latitude": "city_latitude",
                            "data.longitude": "city_longitude",
                            "data.population": "population"},
                 inplace = True)

In [ ]:
city_data

,city_id,city,country,country_code,region,elevation,city_latitude,city_longitude,population
0,Q64,Berlin,Germany,DE,Berlin,None,52.516667,13.383333,3664088
1,Q1055,Hamburg,Germany,DE,Hamburg,None,53.550000,10.000000,1852478
2,Q1726,Munich,Germany,DE,Bavaria,519,48.134520,11.571000,1488202
3,Q365,Cologne,Germany,DE,North Rhine-Westphalia,52,50.942222,6.957778,1083498
4,Q1794,Frankfurt am Main,Germany,DE,Hesse,112,50.113611,8.679722,764104
5,Q1022,Stuttgart,Germany,DE,Baden-Württemberg,245,48.776111,9.177500,630305
6,Q1718,Düsseldorf,Germany,DE,North Rhine-Westphalia,38,51.231144,6.772381,645923
7,Q1295,Dortmund,Germany,DE,North Rhine-Westphalia,86,51.513889,7.465278,587696
8,Q2066,Essen,Germany,DE,North Rhine-Westphalia,116,51.450833,7.013056,582415
9,Q2079,Leipzig,Germany,DE,Saxony,113,51.340000,12.375000,597215


In [ ]:
city_data.dtypes

city_id            object
city               object
country            object
country_code       object
region             object
elevation          object
city_latitude     float64
city_longitude    float64
population          int64
dtype: object

In [ ]:
city_data.to_csv("city_data.csv")